# Chapter 8 recap - Training a model for movie review classification

This section is a recap of the logistic regression model that was trained in the last section of Chapter 6. Execute the following code blocks to train a model that we will serialize in the next section.

**Note**

The code below is based on the `movie_data.csv` dataset that was created in Chapter 8

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/sann-
[nltk_data]     htet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [3]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [4]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [5]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log_loss', random_state=1, max_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [6]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:39


In [7]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.868


In [8]:
clf = clf.partial_fit(X_test, y_test)

# Serializing fitted scikit-learn estimators

After we trained the logistic regression model as shown above, we now save the classifier along woth the stop words, Porter Stemmer, and `HashingVectorizer` as serialized objects to our local disk so that we can use the fitted classifier in our web application later.

In [9]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

In [10]:
%%writefile movieclassifier/vectorizer.py
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(os.path.join(cur_dir, 'pkl_objects', 'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

Overwriting movieclassifier/vectorizer.py


After executing the preceeding code cells, we can now restart the IPython notebook kernel to check if the objects were serialized correctly.

First, change the current Python directory to `movieclassifer`:

In [11]:
import os
os.chdir('movieclassifier')

In [12]:
import pickle
import re
import os
from vectorizer import vect

clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))

After we have successfully loaded the `vectorizer` and unpickled the classifier, we
can use these objects to preprocess document examples and make predictions about
their sentiments:

In [13]:
import numpy as np

label = {0: 'negative', 1: 'positive'}

example = ["I love this movie. It's amazing"]
X = vect.transform(example)
print('Prediction: %s\nProbability: %.2f%%' % (label[clf.predict(X)[0]],
                                               np.max(clf.predict_proba(X))*100))

Prediction: positive
Probability: 95.55%


# Setting up a SQLite database for data storage 

Note that we are still on the "movieclassifier" subdirectory:

In [14]:
os.getcwd()

'/home/sann-htet/Desktop/Python Machine Learning: Machine Learning and Deep Learning with Python, scikit-learn, and TensorFlow/Chapter 9/movieclassifier'

In [15]:
import sqlite3
import os

conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('DROP TABLE IF EXISTS review_db')
c.execute('CREATE TABLE review_db(review TEXT, sentiment INTEGER, date TEXT)')

example1 = 'I love this movie'
c.execute("INSERT INTO review_db(review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example1, 1))

example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db(review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example2, 0))
conn.commit()
conn.close()

In [16]:
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()

c.execute("SELECT * FROM review_db WHERE date BETWEEN '2023-01-01 00:00:00' AND DATETIME('now')")
results = c.fetchall()

conn.close()

In [17]:
print(results)

[('I love this movie', 1, '2023-08-01 12:45:31'), ('I disliked this movie', 0, '2023-08-01 12:45:31')]


# Developing a web application with Flask

## Our first Flask web application

In [18]:
os.chdir('..')

In [19]:
os.listdir()

['1st_flask_app_1',
 'movie_data.csv',
 '.ipynb_checkpoints',
 'movieclassifier',
 '1st_flask_app_2',
 'ch09-Embedding_a_Machine_Learning_Model_into_a_Web_Application.ipynb']

In [20]:
os.chdir('1st_flask_app_1/')

In [21]:
os.listdir()

['templates', '.ipynb_checkpoints', 'app.py']

In [22]:
%%writefile app.py
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('first_app.html')

if __name__ == '__main__':
    app.run()

Overwriting app.py


In [23]:
os.chdir('templates/')

In [24]:
%%writefile first_app.html
<!doctype html>
<html>
    <head>
        <title>First app</title>
    </head>
    <body>
        <div>Hi, this is my first Flask web app!</div>
    </body>
</html>

Overwriting first_app.html


In [25]:
os.chdir('..')

Now, let's start our web application by executing the command from the
terminal inside the `1st_flask_app_1` directory:

In [26]:
!python app.py

/bin/bash: /home/sann-htet/anaconda3/envs/tensorflow_gpu/lib/libtinfo.so.6: no version information available (required by /bin/bash)
 * Serving Flask app 'app'
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
^C


## Form validation and rendering

In [27]:
os.getcwd()

'/home/sann-htet/Desktop/Python Machine Learning: Machine Learning and Deep Learning with Python, scikit-learn, and TensorFlow/Chapter 9/1st_flask_app_1'

In [28]:
os.chdir('../1st_flask_app_2/')

In [29]:
os.listdir()

['static', 'templates', '.ipynb_checkpoints', 'app.py']

In [30]:
%%writefile app.py
from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators

app = Flask(__name__)

class HelloForm(Form):
    sayhello = TextAreaField('', [validators.DataRequired()])
    
@app.route('/')
def index():
    form = HelloForm(request.form)
    return render_template('first_app.html', form=form)

@app.route('/hello', methods=['POST'])
def hello():
    form = HelloForm(request.form)
    if request.method=='POST' and form.validate():
        name = request.form['sayhello']
        return render_template('hello.html', name=name)
    return render_template('first_app.html', form=form)

if __name__ == '__main__':
    app.run(debug=True)

Overwriting app.py


## Implementing a macro using the Jinja2 templating engine

In [31]:
os.chdir('templates/')

In [32]:
%%writefile _formhelpers.html
{% macro render_field(field) %}
    <dt>{{ field.label }}
    <dd>{{ field(**kwargs)|safe }}
    {% if field.errors %}
        <ul class=errors>
        {% for error in field.errors %}
            <li>{{ error }}</li>
        {% endfor %}
        </ul>
    {% endif %}
    </dd>
    </dt>
{% endmacro %}

Overwriting _formhelpers.html


## Adding style via CSS

In [33]:
os.chdir('../static/')

In [34]:
%%writefile style.css
body {
    font-size: 2em;
}

Overwriting style.css


The following are the contents of the modified `first_app.html` file that will now
render a text form where a user can enter a name:

In [35]:
os.chdir('../templates/')

In [36]:
%%writefile first_app.html
<!doctype html>
<html>
    <head>
        <title>First app</title>
            <link rel="stylesheet" href="{{url_for('static', filename='style.css')}}">
    </head>
    <body>
        {% from "_formhelpers.html" import render_field %}
        <div>What's your name?</div>
        <form method=post action="/hello">
            <dl>
                {{ render_field(form.sayhello) }}
            </dl>
            <input type=submit value='Say Hello' name='submit_btn'>
        </form>
    </body>
</html>

Overwriting first_app.html


## Creating the result page

In [37]:
%%writefile hello.html
<!doctype html>
<html>
    <head>
        <title>First app</title>
            <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    </head>
    <body>
        <div>Hello {{ name }}</div>
    </body>
</html>

Overwriting hello.html


Having set up our modified Flask web application, we can run it locally by executing
the following command from the application's main directory:

In [38]:
os.chdir('..')

In [39]:
!python app.py

/bin/bash: /home/sann-htet/anaconda3/envs/tensorflow_gpu/lib/libtinfo.so.6: no version information available (required by /bin/bash)
 * Serving Flask app 'app'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 223-440-534
^C


# Turning the movie classifier into a web application

## Implementing the main application as app.py

In [40]:
os.getcwd()

'/home/sann-htet/Desktop/Python Machine Learning: Machine Learning and Deep Learning with Python, scikit-learn, and TensorFlow/Chapter 9/1st_flask_app_2'

In [41]:
os.chdir('../movieclassifier/')

In [42]:
os.listdir()

['static',
 '__pycache__',
 'reviews.sqlite',
 'pkl_objects',
 'templates',
 '.ipynb_checkpoints',
 'app.py',
 'vectorizer.py']

In [43]:
%%writefile app.py
from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators
import pickle
import sqlite3
import os
import numpy as np

# import HashingVectorizer from local dir
from vectorizer import vect

app = Flask(__name__)

######## Preparing the Classifier
cur_dir = os.path.dirname(__file__)
clf = pickle.load(open(os.path.join(cur_dir,
                 'pkl_objects',
                 'classifier.pkl'), 'rb'))
db = os.path.join(cur_dir, 'reviews.sqlite')

def classify(document):
    label = {0: 'negative', 1: 'positive'}
    X = vect.transform([document])
    y = clf.predict(X)[0]
    proba = np.max(clf.predict_proba(X))
    return label[y], proba

def train(document, y):
    X = vect.transform([document])
    clf.partial_fit(X, [y])

def sqlite_entry(path, document, y):
    conn = sqlite3.connect(path)
    c = conn.cursor()
    c.execute("INSERT INTO review_db (review, sentiment, date)" " VALUES (?, ?, DATETIME('now'))", (document, y))
    conn.commit()
    conn.close()

######## Flask
class ReviewForm(Form):
    moviereview = TextAreaField('',
                                [validators.DataRequired(),
                                 validators.length(min=15)])

@app.route('/')
def index():
    form = ReviewForm(request.form)
    return render_template('reviewform.html', form=form)

@app.route('/results', methods=['POST'])
def results():
    form = ReviewForm(request.form)
    if request.method == 'POST' and form.validate():
        review = request.form['moviereview']
        y, proba = classify(review)
        return render_template('results.html',
                                content=review,
                                prediction=y,
                                probability=round(proba*100, 2))
    return render_template('reviewform.html', form=form)

@app.route('/thanks', methods=['POST'])
def feedback():
    feedback = request.form['feedback_button']
    review = request.form['review']
    prediction = request.form['prediction']

    inv_label = {'negative': 0, 'positive': 1}
    y = inv_label[prediction]
    if feedback == 'Incorrect':
        y = int(not(y))
    train(review, y)
    sqlite_entry(db, review, y)
    return render_template('thanks.html')

if __name__ == '__main__':
    app.run(debug=True)

Overwriting app.py


## Setting up the review form

In [44]:
os.chdir('templates/')

In [45]:
%%writefile reviewform.html
<!doctype html>
<html>
    <head>
        <title>Movie Classification</title>
            <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    </head>
    <body>

        <h2>Please enter your movie review:</h2>

        {% from "_formhelpers.html" import render_field %}

        <form method=post action="/results">
            <dl>
                {{ render_field(form.moviereview, cols='30', rows='10') }}
            </dl>
            <div>
                <input type=submit value='Submit review' name='submit_btn'>
            </div>
        </form>

    </body>
</html>

Overwriting reviewform.html


## Creating a results page template

In [46]:
%%writefile results.html
<!doctype html>
<html>
    <head>
        <title>Movie Classification</title>
            <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    </head>
    <body>
        <h3>Your movie review:</h3>
            
        <div>{{ content }}</div>

        <h3>Prediction:</h3>
        <div>This movie review is <strong>{{ prediction }}</strong>(probability: {{ probability }}%).</div>

        <div id='button'>
            <form action="/thanks" method="post">
                <input type=submit value='Correct' name='feedback_button'>
                <input type=submit value='Incorrect' name='feedback_button'>
                <input type=hidden value='{{ prediction }}' name='prediction'>
                <input type=hidden value='{{ content }}' name='review'>
            </form>
        </div>

        <div id='button'>
            <form action="/">
                <input type=submit value='Submit another review'>
            </form>
        </div>
    </body>
</html>

Overwriting results.html


In [47]:
%%writefile thanks.html
<!doctype html>
<html>
    <head>
        <title>Movie Classification</title>
            <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    </head>
    <body>
    
        <h3>Thank you for your feedback!</h3>

        <div id='button'>
            <form action="/">
                <input type=submit value='Submit another review'>
            </form>
        </div>
    </body>
</html>

Overwriting thanks.html


In [48]:
%%writefile _formhelpers.html
{% macro render_field(field) %}
    <dt>{{ field.label }}
    <dd>{{ field(**kwargs)|safe }}
    {% if field.errors %}
        <ul class=errors>
        {% for error in field.errors %}
            <li>{{ error }}</li>
        {% endfor %}
        </ul>
    {% endif %}
    </dd>
    </dt>
{% endmacro %}

Overwriting _formhelpers.html


In [49]:
os.chdir('../static/')

In [50]:
%%writefile style.css
body{
    width:600px;
}

.button{
    padding-top: 20px;
}

Overwriting style.css


In [51]:
os.chdir('..')

In [52]:
!python app.py

/bin/bash: /home/sann-htet/anaconda3/envs/tensorflow_gpu/lib/libtinfo.so.6: no version information available (required by /bin/bash)
 * Serving Flask app 'app'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 223-440-534
^C


# Deploying the web application to a public server

## Updating the movie classifier application

Define a function to update the classifier with the data stored in the local SQLite database:

In [53]:
import pickle
import sqlite3
import numpy as np

# import HashingVectorizer from local dir
from vectorizer import vect

def update_model(db_path, model, batch_size=10000):

    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute('SELECT * from review_db')
    
    results = c.fetchmany(batch_size)
    while results:
        data = np.array(results)
        X = data[:, 0]
        y = data[:, 1].astype(int)
    
        classes = np.array([0, 1])
        X_train = vect.transform(X)
        clf.partial_fit(X_train, y, classes=classes)
        results = c.fetchmany(batch_size)
    
    conn.close()
    return None

Update the model:

In [54]:
cur_dir = '.'

# Use the following path instead if you embed this code into
# the app.py file

# import os
# cur_dir = os.path.dirname(__file__)

clf = pickle.load(open(os.path.join(cur_dir,
                 'pkl_objects',
                 'classifier.pkl'), 'rb'))
db = os.path.join(cur_dir, 'reviews.sqlite')

update_model(db_path=db, model=clf, batch_size=10000)

# Uncomment the following lines to update your classifier.pkl file

# pickle.dump(clf, open(os.path.join(cur_dir, 
#             'pkl_objects', 'classifier.pkl'), 'wb')
#             , protocol=4)

---